## Блокнот для прикидки влияния отдельных параметров


In [1]:
# from utils import X,y,cat_cols,fillna,normalize,skew,encode
from task import *
import gc
gc.collect()

1131

### Для тестирования используется тренировочный набор
Набор разделён на обучающий и тестовый  в соотношении 3:1 без перемешивания 

In [2]:
def data():
    split = 300_000
    maxlines = 500_000
    d2 = pd.read_csv('datasets/train.csv',skiprows= range(1,split) )
    d1 = pd.read_csv('datasets/train.csv',skiprows= range(split,maxlines) )
    return(d1,d2)  


### Класс для прогона тестов
На экран выводится метрика MAPE на тетсовом наборе 
Кроме того, базовый класс записывает метрику в лог, откуда её можно извлечь

In [3]:
class Validator(Task):

    def __init__( self, data, params = {}):
        super().__init__(data, params)
        self.ds = ScoreSet( data[0],data[1] ,params=self.params)

    def process(self,ds = None ):
        if ds==None:
            ds = self.ds
        ds.set_encoder( self.new_encoder() ).set_model(self.new_model() )
        ds.prepare().fit().predict()
        self.log +=(f"{ds.log}\t MAPE:{ds.scores()}\n")          

    

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>
</div>

> Мне очень нравится качество решения и написанного кода! Но я переживаю за то, что ты используешь ООП слишком часто и не всегда обоснованно. Ведь если быть честным, такую маленькую задачу как выше можно решить с использованием простой функции. Мой искренний совет стараться писать в функциональным стиле программирования, так как на нём как правило пишется весь промышленный код в компании. К классам прибегать когда есть много методов, нужно реализовать наследование и т.д.

### Подбор параметров в цикле

In [5]:
X = ['year','make','model','trim','body','transmission','state','seller','condition','odometer']
cat_cols = ['make','model','trim','body','transmission','state','seller','year','odometer','condition'] 
params = { 
            'rounding': { 'odometer':3000,'condition':1},
            'model':{  'criterion':'poisson','n_estimators':128, 'max_depth':32, 'min_samples_leaf':2, 'max_features':16},
            'slice':{'cat_cols':['make','model','trim','body','transmission','state','seller','year','odometer','condition']}
        }
mc = RandomForestRegressor
ec = TargetEncoder
for odo in [1000,3000,600]:
    for st  in [4,5,6]:
        for ps in [10,12,15]:
            params.update({'rounding': { 'odometer':odo,'condition':1,'prefix_size':ps}, 
 #                          'skew': {'threshold':th,'mult':mult },
                           'model_class': mc, 'encoder_class': ec,    
                           'encoder': {'min_samples_leaf':2,'smoothing':5} ,
                           'sellers':{'stages':st}
                          })
            v = Validator( data() ,params=params)
            v.process()
            print(v.log)            
            

	 MAPE:0.1335834757683081

	 MAPE:0.13357836349610447

	 MAPE:0.13364865749492688

	 MAPE:0.13231045369323488

	 MAPE:0.13205915785455016

	 MAPE:0.1321691282272522

	 MAPE:0.13149478847831372

	 MAPE:0.13204396519648323

	 MAPE:0.13227864388914465

	 MAPE:0.133329046879516

	 MAPE:0.1339745823368105

	 MAPE:0.1348661915155183

	 MAPE:0.13461268963445164

	 MAPE:0.13544794174208244

	 MAPE:0.1362038155302887

	 MAPE:0.13560442976460338

	 MAPE:0.1367956188365341

	 MAPE:0.1374075418680119

	 MAPE:0.14003545305790321

	 MAPE:0.14074428819333681

	 MAPE:0.14225734600275192

	 MAPE:0.14161572035102576

	 MAPE:0.1429499305886468

	 MAPE:0.14390471452893713

	 MAPE:0.14358121163224113

	 MAPE:0.14434289921913215

	 MAPE:0.14476838935045666



лучший результат MAPE:0.13149478847831372 достигнут при 

'rounding': {'odometer': 1000, 'condition': 1, 'prefix_size': 10}, 'model': {'criterion': 'poisson', 'n_estimators': 128, 'max_depth': 32, 'min_samples_leaf': 2, 'max_features': 16}, 'slice': {'cat_cols': ['make', 'model', 'trim', 'body', 'transmission', 'state', 'seller', 'year', 'odometer', 'condition']}, 'model_class': <class 'sklearn.ensemble._forest.RandomForestRegressor'>, 'encoder_class': <class 'category_encoders.target_encoder.TargetEncoder'>, 'encoder': {'min_samples_leaf': 2, 'smoothing': 5}, 'sellers': {'stages': 6}